In [13]:
import pandas as pd
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm 
import re

import re
import requests
import folium
from bs4 import BeautifulSoup as BS

In [14]:
# 식당 데이터 임포트

name_data1 = pd.read_csv("서울관광재단_식당품질정보_20230111 - 금천구.csv", encoding='cp949')
name_data1.rename(columns = {'Unnamed: 0':'식당명',
                            'Unnamed: 1':'지역명',
                            'Unnamed: 2':'네이버평점'}, inplace=True)
name_data1.dropna(axis=0, inplace=True)
name_data1.head()
 

,식당명,지역명,네이버평점
0,오늘 한판,금천구,5.0
1,중한홍원면점,금천구,5.0
2,솔다방,금천구,5.0
3,팔팔식당,금천구,5.0
4,우리식당,금천구,5.0


In [15]:
name_data2 = pd.read_csv("서울관광재단_식당품질정보_20230111 - 동대문구.csv", encoding='cp949')
name_data2.rename(columns = {'Unnamed: 0':'식당명',
                            'Unnamed: 1':'지역명',
                            'Unnamed: 2':'네이버평점'}, inplace=True)
name_data2.dropna(axis=0, inplace=True)
name_data2.head()

,식당명,지역명,네이버평점
0,제일식당,동대문구,5.0
1,바니양념통닭,동대문구,5.0
2,목포식당,동대문구,5.0
3,동원다방,동대문구,5.0
4,빈대떡왕대포,동대문구,5.0


In [16]:
name_data3 = pd.read_csv("서울관광재단_식당품질정보_20230111 - 마포구.csv", encoding='cp949')
name_data3.rename(columns = {'Unnamed: 0':'식당명',
                            'Unnamed: 1':'지역명',
                            'Unnamed: 2':'네이버평점'}, inplace=True)
name_data3.dropna(axis=0, inplace=True)
name_data3.head()

,식당명,지역명,네이버평점
0,젠바,마포구,5.0
1,서울가든호텔 카페1883,마포구,5.0
2,마당호프,마포구,5.0
3,묵을래,마포구,5.0
4,로뎀식당,마포구,5.0


In [17]:
name_data4 = pd.read_csv("서울관광재단_식당품질정보_20230111 - 서초구.csv", encoding='cp949')
name_data4.rename(columns = {'Unnamed: 0':'식당명',
                            'Unnamed: 1':'지역명',
                            'Unnamed: 2':'네이버평점'}, inplace=True)
name_data4.dropna(axis=0, inplace=True)
name_data4.head()

,식당명,지역명,네이버평점
0,작은마을,서초구,5.0
1,우도,서초구,5.0
2,즉석강남떡볶이,서초구,5.0
3,쿡앤펍 플러스 보드게임 콘솔게임,서초구,5.0
4,03수제비칼국수,서초구,5.0


In [18]:
name_data5 = pd.read_csv("서울관광재단_식당품질정보_20230111 - 성동구.csv", encoding='cp949')
name_data5.rename(columns = {'Unnamed: 0':'식당명',
                            'Unnamed: 1':'지역명',
                            'Unnamed: 2':'네이버평점'}, inplace=True)
name_data5.dropna(axis=0, inplace=True)
name_data5.head()

,식당명,지역명,네이버평점
0,풀코스,성동구,5.0
1,충남식당,성동구,5.0
2,실비식당,성동구,5.0
3,무등산닭한마리,성동구,5.0
4,소점식당,성동구,5.0


In [19]:
#지역명이 포함된 식당명을 변수로 지정
items1 = name_data1['식당명']
print(items1[:1000],"\n")

items2 = name_data2['식당명']
print(items2[:1000],"\n")

items3 = name_data3['식당명']
print(items3[:1000],"\n")

items4 = name_data4['식당명']
print(items4[:1000],"\n")

items5 = name_data5['식당명']
print(items5[:1000],"\n")

0         오늘 한판
1        중한홍원면점
2           솔다방
3          팔팔식당
4          우리식당
         ...   
995       오징어나라
996        돼지분식
997     프랑세즈과자점
998    포장마차200호
999     풍년쌈밥보리밥
Name: 식당명, Length: 1000, dtype: object 

0        제일식당
1      바니양념통닭
2        목포식당
3        동원다방
4      빈대떡왕대포
        ...  
995     에이셉피자
996     동마곱창집
997     카페이너프
998      수빈마마
999     원조보신탕
Name: 식당명, Length: 1000, dtype: object 

0                 젠바
1      서울가든호텔 카페1883
2               마당호프
3                묵을래
4               로뎀식당
           ...      
995              형제집
996              비에니
997              씨스루
998         후니지니 마카롱
999               도나
Name: 식당명, Length: 1000, dtype: object 

0                   작은마을
1                     우도
2                즉석강남떡볶이
3      쿡앤펍 플러스 보드게임 콘솔게임
4               03수제비칼국수
             ...        
995                울엄마분식
996                  섬마을
997                 커피정원
998                 공간21
999                 two평
Name: 식당명, Length: 1000, dtype: object 



In [150]:
#잘 안됐음
'''searching = items1[:1000]
n=0
places = []
for i in searching:
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(i)
    headers = {
        "Authorization": "KakaoAK baccb3812bb7865995e93c318b31e10e"
    }
    time.sleep(2)
    places_p = requests.get(url, headers = headers).json()['documents'][0]
    places[n] = places_p
    
    n=n+1

places'''

{'address_name': '서울 강북구 미아동 860-46',
 'category_group_code': 'CE7',
 'category_group_name': '카페',
 'category_name': '음식점 > 카페 > 다방',
 'distance': '',
 'id': '19592511',
 'phone': '',
 'place_name': '솔다방',
 'place_url': 'http://place.map.kakao.com/19592511',
 'road_address_name': '서울 강북구 도봉로8길 70',
 'x': '127.0332161651802',
 'y': '37.611022349347614',
 '오늘 한판': {'address_name': '서울 금천구 독산동 988-8',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 중식',
  'distance': '',
  'id': '471771961',
  'phone': '02-863-5888',
  'place_name': '중한홍원면점',
  'place_url': 'http://place.map.kakao.com/471771961',
  'road_address_name': '서울 금천구 시흥대로138길 33',
  'x': '126.900372593336',
  'y': '37.472743736675'},
 '중한홍원면점': {'address_name': '서울 금천구 독산동 988-8',
  'category_group_code': 'FD6',
  'category_group_name': '음식점',
  'category_name': '음식점 > 중식',
  'distance': '',
  'id': '471771961',
  'phone': '02-863-5888',
  'place_name': '중한홍원면점',
  'place_url': 'http://p

In [11]:
import requests
import time

#함수부
def get_places(items):
    places = []
    for i in items:
        url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(i)
        headers = {
            "Authorization": "KakaoAK baccb3812bb7865995e93c318b31e10e"
        }
        time.sleep(2)
        response = requests.get(url, headers=headers).json()
        if 'documents' in response and len(response['documents']) > 0:
            places_p = response['documents'][0]
            places.append(places_p)
    return places

#실행부
searching = items1[:1000]
places = get_places(searching)
print(places)

[{'address_name': '서울 금천구 가산동 41-11', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 한식 > 육류,고기', 'distance': '', 'id': '16593645', 'phone': '', 'place_name': '오늘한판', 'place_url': 'http://place.map.kakao.com/16593645', 'road_address_name': '서울 금천구 벚꽃로 312', 'x': '126.88298000435', 'y': '37.4822199229043'}, {'address_name': '서울 금천구 독산동 988-8', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 중식', 'distance': '', 'id': '471771961', 'phone': '02-863-5888', 'place_name': '중한홍원면점', 'place_url': 'http://place.map.kakao.com/471771961', 'road_address_name': '서울 금천구 시흥대로138길 33', 'x': '126.900372593336', 'y': '37.472743736675'}, {'address_name': '서울 강북구 미아동 860-46', 'category_group_code': 'CE7', 'category_group_name': '카페', 'category_name': '음식점 > 카페 > 다방', 'distance': '', 'id': '19592511', 'phone': '', 'place_name': '솔다방', 'place_url': 'http://place.map.kakao.com/19592511', 'road_address_name': '서울 강북구 도봉로8길 70', 'x': '127

In [ ]:
import json

# places 리스트를 JSON 파일로 저장
with open('places.json', 'w') as json_file:
    json.dump(places, json_file)

In [12]:
import json

# JSON 파일에서 places 리스트 불러오기
with open('places.json', 'r') as json_file:
    loaded_places = json.load(json_file)

print(loaded_places)  # 불러온 데이터 확인

[{'address_name': '서울 금천구 가산동 41-11', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 한식 > 육류,고기', 'distance': '', 'id': '16593645', 'phone': '', 'place_name': '오늘한판', 'place_url': 'http://place.map.kakao.com/16593645', 'road_address_name': '서울 금천구 벚꽃로 312', 'x': '126.88298000435', 'y': '37.4822199229043'}, {'address_name': '서울 금천구 독산동 988-8', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 중식', 'distance': '', 'id': '471771961', 'phone': '02-863-5888', 'place_name': '중한홍원면점', 'place_url': 'http://place.map.kakao.com/471771961', 'road_address_name': '서울 금천구 시흥대로138길 33', 'x': '126.900372593336', 'y': '37.472743736675'}, {'address_name': '서울 강북구 미아동 860-46', 'category_group_code': 'CE7', 'category_group_name': '카페', 'category_name': '음식점 > 카페 > 다방', 'distance': '', 'id': '19592511', 'phone': '', 'place_name': '솔다방', 'place_url': 'http://place.map.kakao.com/19592511', 'road_address_name': '서울 강북구 도봉로8길 70', 'x': '127

In [20]:
searching = items2[:1000]
places2 = get_places(searching)
print(places2)

with open('places2.json', 'w') as json_file:
    json.dump(places2, json_file)
    

searching = items3[:1000]
places3 = get_places(searching)
print(places3)

with open('places3.json', 'w') as json_file:
    json.dump(places3, json_file)

    
searching = items4[:1000]
places4 = get_places(searching)
print(places4)

with open('places4.json', 'w') as json_file:
    json.dump(places4, json_file)
    

searching = items5[:1000]
places5 = get_places(searching)
print(places5)

with open('places5.json', 'w') as json_file:
    json.dump(places5, json_file)

[{'address_name': '경남 진주시 대안동 8-49', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 한식', 'distance': '', 'id': '8068808', 'phone': '055-741-5591', 'place_name': '제일식당', 'place_url': 'http://place.map.kakao.com/8068808', 'road_address_name': '경남 진주시 중앙시장길 37-8', 'x': '128.08474757723928', 'y': '35.1951020207355'}, {'address_name': '서울 동대문구 휘경동 312-261', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 치킨', 'distance': '', 'id': '24183751', 'phone': '02-248-9124', 'place_name': '바니양념통닭', 'place_url': 'http://place.map.kakao.com/24183751', 'road_address_name': '서울 동대문구 망우로 14', 'x': '127.055048570895', 'y': '37.5876352749316'}, {'address_name': '전남 함평군 함평읍 기각리 981-8', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 한식 > 육류,고기', 'distance': '', 'id': '7875961', 'phone': '061-322-2764', 'place_name': '목포식당', 'place_url': 'http://place.map.kakao.com/7875961', 'road_address_name': '전남 함

[{'address_name': '서울 마포구 서교동 358-46', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 술집 > 칵테일바', 'distance': '', 'id': '15697426', 'phone': '02-325-8540', 'place_name': '젠', 'place_url': 'http://place.map.kakao.com/15697426', 'road_address_name': '서울 마포구 와우산로21길 34', 'x': '126.92216246555384', 'y': '37.552471020514524'}, {'address_name': '서울 종로구 관철동 155', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 술집 > 호프,요리주점', 'distance': '', 'id': '22247789', 'phone': '02-720-0088', 'place_name': '마당호프', 'place_url': 'http://place.map.kakao.com/22247789', 'road_address_name': '서울 종로구 청계천로 67', 'x': '126.985108247882', 'y': '37.5688380180224'}, {'address_name': '경북 김천시 신음동 1015-1', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 한식 > 육류,고기 > 닭요리', 'distance': '', 'id': '309975915', 'phone': '054-910-7770', 'place_name': '닭발묵을래', 'place_url': 'http://place.map.kakao.com/309975915', 'road_

[{'address_name': '전남 고흥군 포두면 오취리', 'category_group_code': '', 'category_group_name': '', 'category_name': '부동산 > 지명', 'distance': '', 'id': '25346473', 'phone': '', 'place_name': '작은마을', 'place_url': 'http://place.map.kakao.com/25346473', 'road_address_name': '', 'x': '127.424388242536', 'y': '34.5496818897182'}, {'address_name': '제주특별자치도 제주시 우도면 연평리', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 섬', 'distance': '', 'id': '8025182', 'phone': '064-728-4333', 'place_name': '우도', 'place_url': 'http://place.map.kakao.com/8025182', 'road_address_name': '', 'x': '126.955582257582', 'y': '33.5044199749505'}, {'address_name': '서울 강남구 역삼동 827-1', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 분식 > 떡볶이', 'distance': '', 'id': '27281065', 'phone': '010-8848-2586', 'place_name': '반장즉석떡볶이 강남점', 'place_url': 'http://place.map.kakao.com/27281065', 'road_address_name': '서울 강남구 테헤란로4길 25', 'x': '127.03046619859454', 'y

[{'address_name': '서울 송파구 방이동 192-4', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 일식', 'distance': '', 'id': '19977014', 'phone': '02-422-3349', 'place_name': '다다풀코스', 'place_url': 'http://place.map.kakao.com/19977014', 'road_address_name': '서울 송파구 위례성대로16길 4-21', 'x': '127.123917393847', 'y': '37.5126777039635'}, {'address_name': '경남 거제시 고현동 98-27', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 한식 > 순대', 'distance': '', 'id': '10087211', 'phone': '055-632-1332', 'place_name': '충남식당', 'place_url': 'http://place.map.kakao.com/10087211', 'road_address_name': '경남 거제시 거제중앙로 1883-2', 'x': '128.6236008963253', 'y': '34.886416616640055'}, {'address_name': '서울 중구 인현동2가 210-2', 'category_group_code': 'FD6', 'category_group_name': '음식점', 'category_name': '음식점 > 한식', 'distance': '', 'id': '908930966', 'phone': '02-2269-3860', 'place_name': '실비식당', 'place_url': 'http://place.map.kakao.com/908930966', 'road_address_name':